## Beam Sensitivity

Perform sensitivity analysis on beams

In progress - working on absorption function and comparing two methods for calculation.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import importlib
import tri_mesh
import beam
import beam_utils
from scipy import interpolate

from lmfit.lineshapes import gaussian, voigt
import cross_section

## Read Jet Data

In [ ]:
viz_path = r"C:\Users\Huckaby\Desktop\MHD\Simulations\viz_example" 
print(os.listdir(viz_path))
fname = "frontCyl_plasma.vtk"
fname = "frontCyl.vtk"
fname = os.path.join(viz_path, fname)
f_flow = beam_utils.new_soln_interpolator(fname, do_plot=False)

In [ ]:
#source

## Source Wavelength Profile

In [ ]:
importlib.reload(beam_utils)
importlib.reload(cross_section)
#dir(beam_uilts)

In [ ]:
data_path = r"C:\Users\Huckaby\Desktop\MHD\Simulations\viz_example"
fname = "ds_calib.cdf"
fname = os.path.join(data_path,fname)
source_wavelength = beam_utils.read_intensity_wavelength_function(fname, do_plot=False)
source_wavelength

In [ ]:
source_filt = beam_utils.filter_wave_profile(source_wavelength)
mp = "mw_horn"
fig, ax = plt.subplots(2,1,sharex=True)
for i in range(2):
    source_wavelength["diff"].isel(time=0,mp=i).plot(ax=ax[i])
    source_filt["diff"].isel(time=0,mp=i).plot(ax=ax[i])
fig.tight_layout()

## Beam

In [ ]:
importlib.reload(tri_mesh)
importlib.reload(beam)

In [ ]:
def mag(x):
    return np.sum(x**2, axis=-1)**0.5

In [ ]:
x_exit = 207.910 * 1e-3

# location of source relative to focus point
source_offset = np.array([-0.05,0.0,-0.1])
source_distance = mag(source_offset)
beam_dir = -source_offset/source_distance

target_distance = source_distance
target_offset = target_distance*beam_dir
print("target distance = {:8.2f} [cm]".format(target_distance*1e2))
print("source distance = {:8.2f} [cm]".format(source_distance*1e2))
# minumum
x_min = x_exit - source_offset[0]
x_focus = x_min
pos_focus = np.array([x_focus,0,0])

b = beam.Beam(source=pos_focus+source_offset,target=pos_focus+target_offset,diam=1e-2)

In [ ]:
b.mesh(n_s=10,n_r=4,n_sides=6)



### Interpolate Field Data to Beam

In [ ]:
f_out = f_flow(b.ds["pos"])
b.ds["T"] = ("s","ray"), f_out[:,:,0].T
b.ds["K"] = ("s","ray"), f_out[:,:,1].T
fig, ax = plt.subplots(2,1)
ax[0].set_aspect(1.0)
#ax[1].set_aspect(1.0)
b.ds["T"].plot(x="z",y="x", ax=ax[0])
b.ds["T"].plot(x="x_beam",y="z_beam", ax=ax[1])
fig.tight_layout()

### Refine Mesh

In [ ]:
import beam
importlib.reload(beam)
beam_adapt = beam.adapt_beam_dataset(b.ds, adapt_vars=["T","K"], field_func=f_flow, rtol=0.1, max_iter=20)
b.add_cart_coords(beam_adapt, "pos_beam", "{}_beam")

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].set_aspect(1.0)
#ax[1].set_aspect(1.0)
beam_adapt["T"].plot(x="z",y="x", ax=ax[0])
beam_adapt["T"].plot(x="x_beam",y="z_beam", ax=ax[1])
fig.tight_layout()

In [ ]:
rays = [0,20]
n_rays = len(rays)
fig, ax = plt.subplots(1,n_rays,sharey=True)
for i, ray in enumerate(rays):
    beam_adapt["T"].sel(ray=ray).plot(label="adapt", ax=ax[i])
    b.ds["T"].sel(ray=ray).plot(label="coarse", ax=ax[i])
    pos = beam_adapt["pos"].sel(ray=ray)
    f = f_flow(pos)
    ax[i].plot(beam_adapt["s"], f[:,0], 'o', alpha=0.5, label="interp")
    ax[i].legend()
fig.set_size_inches(11,4)
ax[0].set_ylabel("T[K]")

## Calculate Intensity

In [ ]:
#constants.Boltzmann, constants.R/constants.Avogadro

In [ ]:
#Q_absorption.T0

### Potassium Cross Section

In [ ]:
import ray_integration
import scipy.constants as constants
importlib.reload(cross_section)
A0 = constants.e/(4.0*constants.epsilon_0*constants.m_e*constants.c**2)
lam_nm = np.array([766.504333,769.913219])
unit_nm = 1e-9
lam = lam_nm * unit_nm

f = np.array([ 0.7, 0.35 ])
A = A0*np.array([ 4.0/6.0*f[0]*lam[0]**2, f[1]*lam[1]**2 ])

MW_K = 39.1
mass_K = MW_K*constants.m_p
T_ref = 1000.0
k_B = constants.Boltzmann
f_delta_C = [3.19e9, 3.04e9]
delta_D = 2.0*np.sqrt(2.0*np.log(2.0)*k_B*T_ref/mass_K)*(lam/constants.c)
delta_C = (lam**2/constants.c)*f_delta_C

print("Broadening")
print("Doppler [nm]",delta_D/unit_nm)
print("Collisional [nm]", delta_C/unit_nm)
print("lam/c", lam/constants.c)
print()

Q_absorption = cross_section.GaussAbsorption(A=A,b=[-0.5,-0.5], lam=lam_nm, w=[5,5])
Q_absorption.Q0 = 1.0

In [ ]:
unit_cm = 1e-2
E = 13042/unit_cm
E*constants.c

In [ ]:
T = 1000.0
nu = np.sqrt(8.0*(constants.Boltzmann*T*np.log(2.0)/(constants.m_e*constants.c**2)))
lam = 770.0*unit_nm
nu_0 = 1.0/lam
nu_0/1e-2

8

In [ ]:
Q_abs = cross_section.VoigtAbsorption()
lam = np.linspace(761, 780, 100)
T = np.arange(500,3001,500)
#T = np.array([999,1001])
sigma = 3e-3*(T/1000.0)**0.5
gamma = 6e-3*5e2
center = 766.504333
#
# compare new function with older
# broken need to fix
#val = np.array([Q_abs.calc(lam, 1.0, center, sigma_i, gamma)/voigt(center, 1.0, center, sigma_i, gamma) for sigma_i in sigma])
#for i, t  in enumerate(T):
#    plt.plot(lam, val[i], label=t)
#plt.legend()

Q_abs.calc(T=1000.0, do_plot=True)

In [ ]:
cross_section.blurred_alpha(1.0)

In [ ]:
for T in [1000,1500,2000]:
    wave_nm = source_wavelength.wavelength.data
    Q = Q_absorption(T, wave_nm)
    plt.plot(wave_nm, Q, label=T)
plt.legend()

In [ ]:
#from lmfit.lineshapes import gaussian, voigt

In [ ]:
# voigt(x, amplitude=1.0, center=0.0, sigma=1.0, gamma=None)
# kappa = p1_prefactor*voigt(x, 1, p1_center, p1_delta_wl_D, p1_delta_wl_C) + p2_prefactor*voigt(x, 1, p2_center, p2_delta_wl_D, p2_delta_wl_C)